In [1]:
import json

from langchain.chains.question_answering import load_qa_chain

from langchain_community.llms import Ollama
from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatMaritalk

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.text_splitter import RecursiveCharacterTextSplitter

import numpy as np


In [2]:
# obtains the API key used to access Maritalk's API
with open('API_KEY.json', 'r') as file:
    maritalk = json.load(file)

model = ChatMaritalk(
    model='sabia-3',
    api_key=maritalk['key'],
    temperature=0.7,
    max_tokens=500,
)

In [3]:
ollama_base_url='http://localhost:11434' #http://tempestade.facom.ufms.br:11435'
model = Ollama(base_url=ollama_base_url, model="phi3:medium")

# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [4]:
embeddings_model = LaserEmbeddings(lang='por_Latn')

2024-09-09 07:16:42,071 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-09-09 07:16:42,910 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-09-09 07:16:43,074 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2024-09-09 07:16:43,075 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-09-09 07:16:43,076 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


In [8]:
doc_path = './database/edital-1.pdf'

pdf_loader = PyPDFLoader(doc_path)

loaded_pdf = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        separators=['\n', ' ']
)

pages = text_splitter.split_documents(loaded_pdf)

In [9]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

vectorstore.save_local('database/indexes/document_index')

2024-09-09 07:17:51,034 | INFO | faiss.loader | Loading faiss with AVX2 support.
2024-09-09 07:17:51,283 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


In [24]:
# loads vectorstore from disk
vectorstore = FAISS.load_local('database/indexes/document_index', embeddings_model, allow_dangerous_deserialization=True)

In [25]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

qa_prompt = ChatPromptTemplate.from_messages([
        ('system', '''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:'''),
        ('human', '''Como será a lista de espera?'''),
        ('ai', '''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        ('human', '''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        ('ai', '''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        ('human', prompt_structure)
])

chain = load_qa_chain(model, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [26]:
query = input()

In [27]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10)

In [13]:
# enriching the retrieval result 

# get the page index of the page with the most similar chunk
page_idx = retrieval_result[0][0].metadata['page']

retrieval_result = np.concatenate(
    (
        [x for x in pages if x.metadata['page'] == page_idx],
        [page[0] for page in retrieval_result]),
    axis=0
)

In [28]:
# retrieved context, no enrichment
retrieval_result= [page[0] for page in retrieval_result]

In [29]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?
AI: De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.
Human: 
        Baseado nos seguintes documentos:
        https://sei.ufms.br/sei/controlador.ph